# Probabilistic graphical models

Update examples from

https://d-k-e.github.io/graphical-models/html/d3/db0/md_graphical-models_docs_mdpages_usage.html

First download
https://github.com/D-K-E/graphical-models/tree/master
Then `pip install .`

# PGModel


In [3]:
import pygmodels
import numpy as np

In [4]:
# Example adapted from Darwiche 2009, p. 140
# A -> B -> C


from pygmodels.pgmtype.pgmodel import PGModel
from pygmodels.gtype.edge import Edge, EdgeType
from pygmodels.pgmtype.factor import Factor
from pygmodels.pgmtype.randomvariable import NumCatRVariable

# Example adapted from Darwiche 2009, p. 140
idata = {
    "a": {"outcome-values": [True, False]},
    "b": {"outcome-values": [True, False]},
    "c": {"outcome-values": [True, False]},
}
a = NumCatRVariable(node_id="a", input_data=idata["a"], marginal_distribution=lambda x: 0.6 if x else 0.4)
b = NumCatRVariable(node_id="b", input_data=idata["b"], marginal_distribution=lambda x: 0.5 if x else 0.5)
c = NumCatRVariable(node_id="c", input_data=idata["c"], marginal_distribution=lambda x: 0.5 if x else 0.5)
ab = Edge(
    edge_id="ab",
    edge_type=EdgeType.UNDIRECTED,
    start_node=a,
    end_node=b,
)
bc = Edge(
    edge_id="bc",
    edge_type=EdgeType.UNDIRECTED,
    start_node=b,
    end_node=c,
)


def phi_ba(scope_product):
    """"""
    ss = set(scope_product)
    if ss == set([("a", True), ("b", True)]):
        return 0.9
    elif ss == set([("a", True), ("b", False)]):
        return 0.1
    elif ss == set([("a", False), ("b", True)]):
        return 0.2
    elif ss == set([("a", False), ("b", False)]):
        return 0.8
    else:
        raise ValueError("product error")


def phi_cb(scope_product):
    """"""
    ss = set(scope_product)
    if ss == set([("c", True), ("b", True)]):
        return 0.3
    elif ss == set([("c", True), ("b", False)]):
        return 0.5
    elif ss == set([("c", False), ("b", True)]):
        return 0.7
    elif ss == set([("c", False), ("b", False)]):
        return 0.5
    else:
        raise ValueError("product error")


def phi_a(scope_product):
    s = set(scope_product)
    if s == set([("a", True)]):
        return 0.6
    elif s == set([("a", False)]):
        return 0.4
    else:
        raise ValueError("product error")


ba_f = Factor(gid="ba", scope_vars=set([b, a]), factor_fn=phi_ba)
cb_f = Factor(gid="cb", scope_vars=set([c, b]), factor_fn=phi_cb)
a_f = Factor(gid="a", scope_vars=set([a]), factor_fn=phi_a)
pgm = PGModel(
    gid="pgm",
    nodes=set([a, b, c]),
    edges=set([ab, bc]),
    factors=set([ba_f, cb_f, a_f]),
)
evidences = set([("a", True)])
queries = set([c])
product_factor, a = pgm.cond_prod_by_variable_elimination(queries, evidences)
val = round(product_factor.phi_normal(set([("c", True)])), 4)
expected = 0.32
print(val, expected)
assert np.allclose(val, expected)

0.32 0.32


# Bayesian network

In [5]:
# Bayesian network


from pygmodels.pgmodel.bayesian import BayesianNetwork
from pygmodels.gtype.edge import Edge, EdgeType
from pygmodels.pgmtype.factor import Factor
from pygmodels.pgmtype.randomvariable import NumCatRVariable

# data and nodes
idata = {"outcome-values": [True, False]}

C = NumCatRVariable(node_id="C", input_data=idata, marginal_distribution=lambda x: 0.5)
E = NumCatRVariable(node_id="E", input_data=idata, marginal_distribution=lambda x: 0.5)
F = NumCatRVariable(node_id="F", input_data=idata, marginal_distribution=lambda x: 0.5)
D = NumCatRVariable(node_id="D", input_data=idata, marginal_distribution=lambda x: 0.5)

# edges
CE = Edge(
    edge_id="CE",
    start_node=C,
    end_node=E,
    edge_type=EdgeType.DIRECTED,
)
ED = Edge(
    edge_id="ED",
    start_node=E,
    end_node=D,
    edge_type=EdgeType.DIRECTED,
)
EF = Edge(
    edge_id="EF",
    start_node=E,
    end_node=F,
    edge_type=EdgeType.DIRECTED,
)

# define factor functions


def phi_c(scope_product):
    ss = set(scope_product)
    if ss == set([("C", True)]):
        return 0.8
    elif ss == set([("C", False)]):
        return 0.2
    else:
        raise ValueError("scope product unknown")


def phi_ec(scope_product):
    ss = set(scope_product)
    if ss == set([("C", True), ("E", True)]):
        return 0.9
    elif ss == set([("C", True), ("E", False)]):
        return 0.1
    elif ss == set([("C", False), ("E", True)]):
        return 0.7
    elif ss == set([("C", False), ("E", False)]):
        return 0.3
    else:
        raise ValueError("scope product unknown")


def phi_fe(scope_product):
    ss = set(scope_product)
    if ss == set([("E", True), ("F", True)]):
        return 0.9
    elif ss == set([("E", True), ("F", False)]):
        return 0.1
    elif ss == set([("E", False), ("F", True)]):
        return 0.5
    elif ss == set([("E", False), ("F", False)]):
        return 0.5
    else:
        raise ValueError("scope product unknown")


def phi_de(scope_product):
    ss = set(scope_product)
    if ss == set([("E", True), ("D", True)]):
        return 0.7
    elif ss == set([("E", True), ("D", False)]):
        return 0.3
    elif ss == set([("E", False), ("D", True)]):
        return 0.4
    elif ss == set([("E", False), ("D", False)]):
        return 0.6
    else:
        raise ValueError("scope product unknown")


# instantiate factors with given factor function and implied random variables
CE_f = Factor(gid="CE_f", scope_vars=set([C, E]), factor_fn=phi_ec)
C_f = Factor(gid="C_f", scope_vars=set([C]), factor_fn=phi_c)
FE_f = Factor(gid="FE_f", scope_vars=set([F, E]), factor_fn=phi_fe)
DE_f = Factor(gid="DE_f", scope_vars=set([D, E]), factor_fn=phi_de)
bayes_n = BayesianNetwork(
    gid="ba",
    nodes=set([C, E, D, F]),
    edges=set([EF, CE, ED]),
    factors=set([C_f, DE_f, CE_f, FE_f]),
)
query_vars = set([E])
evidences = set([("F", True)])
probs, alpha = bayes_n.cond_prod_by_variable_elimination(query_vars, evidences=evidences)
query_value = set([("E", True)])
round(probs.phi(query_value), 4)
# 0.774

0.774

# Markov network

In [6]:
# Markov network
# Example from Koller and Friedman, p104
# A - B
# |   |
# C - D

from pygmodels.pgmodel.markov import MarkovNetwork
from pygmodels.gtype.edge import Edge, EdgeType
from pygmodels.pgmtype.factor import Factor
from pygmodels.pgmtype.randomvariable import NumCatRVariable


def make_markov_net():

    # define data and random variable nodes
    idata = {
        "A": {"outcome-values": [True, False]},
        "B": {"outcome-values": [True, False]},
        "C": {"outcome-values": [True, False]},
        "D": {"outcome-values": [True, False]},
    }

    A = NumCatRVariable(node_id="A", input_data=idata["A"], marginal_distribution=lambda x: 0.5)
    B = NumCatRVariable(node_id="B", input_data=idata["B"], marginal_distribution=lambda x: 0.5)
    C = NumCatRVariable(node_id="C", input_data=idata["C"], marginal_distribution=lambda x: 0.5)
    D = NumCatRVariable(node_id="D", input_data=idata["D"], marginal_distribution=lambda x: 0.5)

    # define edges
    AB = Edge(
        edge_id="AB",
        edge_type=EdgeType.UNDIRECTED,
        start_node=A,
        end_node=B,
    )
    AD = Edge(
        edge_id="AD",
        edge_type=EdgeType.UNDIRECTED,
        start_node=A,
        end_node=D,
    )
    DC = Edge(
        edge_id="DC",
        edge_type=EdgeType.UNDIRECTED,
        start_node=D,
        end_node=C,
    )
    BC = Edge(
        edge_id="BC",
        edge_type=EdgeType.UNDIRECTED,
        start_node=B,
        end_node=C,
    )

    # define factor functions

    def phi_AB(scope_product):
        """"""
        ss = frozenset(scope_product)
        if ss == frozenset([("A", False), ("B", False)]):
            return 30.0
        elif ss == frozenset([("A", False), ("B", True)]):
            return 5.0
        elif ss == frozenset([("A", True), ("B", False)]):
            return 1.0
        elif ss == frozenset([("A", True), ("B", True)]):
            return 10.0
        else:
            raise ValueError("product error")

    def phi_BC(scope_product):
        """"""
        ss = frozenset(scope_product)
        if ss == frozenset([("B", False), ("C", False)]):
            return 100.0
        elif ss == frozenset([("B", False), ("C", True)]):
            return 1.0
        elif ss == frozenset([("B", True), ("C", False)]):
            return 1.0
        elif ss == frozenset([("B", True), ("C", True)]):
            return 100.0
        else:
            raise ValueError("product error")

    def phi_CD(scope_product):
        """"""
        ss = frozenset(scope_product)
        if ss == frozenset([("C", False), ("D", False)]):
            return 1.0
        elif ss == frozenset([("C", False), ("D", True)]):
            return 100.0
        elif ss == frozenset([("C", True), ("D", False)]):
            return 100.0
        elif ss == frozenset([("C", True), ("D", True)]):
            return 1.0
        else:
            raise ValueError("product error")

    def phi_DA(scope_product):
        """"""
        ss = frozenset(scope_product)
        if ss == frozenset([("D", False), ("A", False)]):
            return 100.0
        elif ss == frozenset([("D", False), ("A", True)]):
            return 1.0
        elif ss == frozenset([("D", True), ("A", False)]):
            return 1.0
        elif ss == frozenset([("D", True), ("A", True)]):
            return 100.0
        else:
            raise ValueError("product error")

    # instantiate factors with factor functions and implied
    # random variables in scope

    AB_f = Factor(gid="ab_f", scope_vars=set([A, B]), factor_fn=phi_AB)
    BC_f = Factor(gid="bc_f", scope_vars=set([B, C]), factor_fn=phi_BC)
    CD_f = Factor(gid="cd_f", scope_vars=set([C, D]), factor_fn=phi_CD)
    DA_f = Factor(gid="da_f", scope_vars=set([D, A]), factor_fn=phi_DA)

    # instantiate markov network and make a query
    mnetwork = MarkovNetwork(
        gid="mnet",
        nodes=set([A, B, C, D]),
        edges=set([AB, AD, BC, DC]),
        factors=set([DA_f, CD_f, BC_f, AB_f]),
    )

    return mnetwork, A, B, C, D

In [28]:
mnetwork, A, B, C, D = make_markov_net()

queries = set([A, B])
evidences = set()
prob, a = mnetwork.cond_prod_by_variable_elimination(queries, evidences)
q2 = set([("A", False), ("B", True)])
round(prob.phi_normal(q2), 2)
# 0.69

0.69

In [29]:
mnetwork, A, B, C, D = make_markov_net()


# p(B=1)=0.26
queries = set([B])
evidences = set()
prob, a = mnetwork.cond_prod_by_variable_elimination(queries, evidences)
q2 = set([("B", True)])
val = round(prob.phi_normal(q2), 2)
expected = 0.74  # 0.26 # Typo in book?
print(val, expected)
assert np.allclose(val, expected)

0.74 0.74


In [30]:
mnetwork, A, B, C, D = make_markov_net()

# p(B=1|C=0)=0.06
queries = set([B])
evidences = set([("C", False)])
prob, a = mnetwork.cond_prod_by_variable_elimination(queries, evidences)
q2 = set([("B", True)])
val = round(prob.phi_normal(q2), 2)
expected = 0.06
print(val, expected)
assert np.allclose(val, expected)

0.06 0.06


# Conditional random field

In [8]:
from pygmodels.pgmtype.randomvariable import NumCatRVariable
from pygmodels.pgmodel.markov import ConditionalRandomField
from pygmodels.gtype.edge import Edge, EdgeType
from pygmodels.pgmtype.factor import Factor
import math
from random import choice

# Example from Koller ad Friedman p144
# phi(xi, y) = exp(wi * ind(xii=1, y=1))
# phi(y) = exp(w0 * ind(y=1))


# define data and nodes
idata = {"A": {"outcome-values": [True, False]}}

# from Koller, Friedman 2009, p. 144-145, example 4.20
X_1 = NumCatRVariable(node_id="X_1", input_data=idata["A"], marginal_distribution=lambda x: 0.5)
X_2 = NumCatRVariable(node_id="X_2", input_data=idata["A"], marginal_distribution=lambda x: 0.5)
X_3 = NumCatRVariable(node_id="X_3", input_data=idata["A"], marginal_distribution=lambda x: 0.5)
Y_1 = NumCatRVariable(node_id="Y_1", input_data=idata["A"], marginal_distribution=lambda x: 0.5)

# define edges

X1_Y1 = Edge(
    edge_id="X1_Y1",
    edge_type=EdgeType.UNDIRECTED,
    start_node=X_1,
    end_node=Y_1,
)
X2_Y1 = Edge(
    edge_id="X2_Y1",
    edge_type=EdgeType.UNDIRECTED,
    start_node=X_2,
    end_node=Y_1,
)
X3_Y1 = Edge(
    edge_id="X3_Y1",
    edge_type=EdgeType.UNDIRECTED,
    start_node=X_3,
    end_node=Y_1,
)

# define factor functions


def phi_X1_Y1(scope_product):
    """"""
    w = 0.5
    ss = frozenset(scope_product)
    if ss == frozenset([("X_1", True), ("Y_1", True)]):
        return math.exp(1.0 * w)
    else:
        return math.exp(0.0)


def phi_X2_Y1(scope_product):
    """"""
    w = 5.0
    ss = frozenset(scope_product)
    if ss == frozenset([("X_2", True), ("Y_1", True)]):
        return math.exp(1.0 * w)
    else:
        return math.exp(0.0)


def phi_X3_Y1(scope_product):
    """"""
    w = 9.4
    ss = frozenset(scope_product)
    if ss == frozenset([("X_3", True), ("Y_1", True)]):
        return math.exp(1.0 * w)
    else:
        return math.exp(0.0)


def phi_Y1(scope_product):
    """"""
    w = 0.6
    ss = frozenset(scope_product)
    if ss == frozenset([("Y_1", True)]):
        return math.exp(1.0 * w)
    else:
        return math.exp(0.0)


# instantiate factors with factor functions and implied random variables
X1_Y1_f = Factor(gid="x1_y1_f", scope_vars=set([X_1, Y_1]), factor_fn=phi_X1_Y1)
X2_Y1_f = Factor(gid="x2_y1_f", scope_vars=set([X_2, Y_1]), factor_fn=phi_X2_Y1)
X3_Y1_f = Factor(gid="x3_y1_f", scope_vars=set([X_3, Y_1]), factor_fn=phi_X3_Y1)
Y1_f = Factor(gid="y1_f", scope_vars=set([Y_1]), factor_fn=phi_Y1)


# Instantiate conditional random field and make a query
crf_koller = ConditionalRandomField(
    "crf",
    observed_vars=set([X_1, X_2, X_3]),
    target_vars=set([Y_1]),
    edges=set([X1_Y1, X2_Y1, X3_Y1]),
    factors=set([X1_Y1_f, X2_Y1_f, X3_Y1_f, Y1_f]),
)


evidence = set([("Y_1", False)])
query_vars = set([X_1, X_2, X_3])
query = frozenset(
    [
        ("X_1", choice([False, True])),
        ("X_2", choice([False, True])),
        ("X_3", choice([False, True])),
    ]
)
print(query)
out, a1 = crf_koller.cond_prod_by_variable_elimination(queries=query_vars, evidences=evidence)
val = out.phi(query)
expected = 1.0
print(val, expected)
assert np.allclose(val, expected)

frozenset({('X_2', False), ('X_1', True), ('X_3', False)})
1.0 1.0


# Chain graph

In [10]:
# Chain graph
# Based on Cowell et al, 1999
# "Probabilistic Networks and Expert Systems"
# https://link.springer.com/book/10.1007/b97670
# p110
# For a diagram, see https://gist.github.com/murphyk/e157531793f87f3d1eb9542234e6bc7d

# Parameters are same as directed Asia network on p21
# except for the chain component p(C,D | B,E) = phi(C,D,B) phi(C,B,E) 1/Z(B,E)
# where Z(B,E) is derived by normalizing the numerator and C=cough.
# In the code below, these variables are renamed as p(H,I|B,D) = phi(H,I,B) phi(H,B,D) phi(B,D)
# where H=Cough, I=Dysponea, B=Bronchitis, D=Either.

# In the book, the evidence is (a, b, xbar) which in the notation of this code is
# (E=True, A=True, G=False). The query variable is B.

from pygmodels.pgmodel.lwfchain import LWFChainGraph
from pygmodels.gtype.edge import Edge, EdgeType
from pygmodels.pgmtype.factor import Factor
from pygmodels.pgmtype.randomvariable import NumCatRVariable


# define data and nodes
idata = {"outcome-values": [True, False]}
A = NumCatRVariable(node_id="A", input_data=idata, marginal_distribution=lambda x: 0.5)
B = NumCatRVariable(node_id="B", input_data=idata, marginal_distribution=lambda x: 0.5)
C = NumCatRVariable(node_id="C", input_data=idata, marginal_distribution=lambda x: 0.5)
D = NumCatRVariable(node_id="D", input_data=idata, marginal_distribution=lambda x: 0.5)
E = NumCatRVariable(node_id="E", input_data=idata, marginal_distribution=lambda x: 0.5)
F = NumCatRVariable(node_id="F", input_data=idata, marginal_distribution=lambda x: 0.5)
G = NumCatRVariable(node_id="G", input_data=idata, marginal_distribution=lambda x: 0.5)
H = NumCatRVariable(node_id="H", input_data=idata, marginal_distribution=lambda x: 0.5)
I = NumCatRVariable(node_id="I", input_data=idata, marginal_distribution=lambda x: 0.5)
K = NumCatRVariable(node_id="K", input_data=idata, marginal_distribution=lambda x: 0.5)
L = NumCatRVariable(node_id="L", input_data=idata, marginal_distribution=lambda x: 0.5)


AB_c = Edge(
    edge_id="AB",
    start_node=A,
    end_node=B,
    edge_type=EdgeType.DIRECTED,
)
AC_c = Edge(
    edge_id="AC",
    start_node=A,
    end_node=C,
    edge_type=EdgeType.DIRECTED,
)
CD_c = Edge(
    edge_id="CD",
    start_node=C,
    end_node=D,
    edge_type=EdgeType.DIRECTED,
)
EF_c = Edge(
    edge_id="EF",
    start_node=E,
    end_node=F,
    edge_type=EdgeType.DIRECTED,
)
FD_c = Edge(
    edge_id="FD",
    start_node=F,
    end_node=D,
    edge_type=EdgeType.DIRECTED,
)
DG_c = Edge(
    edge_id="DG",
    start_node=D,
    end_node=G,
    edge_type=EdgeType.DIRECTED,
)
DH_c = Edge(
    edge_id="DH",
    start_node=D,
    end_node=H,
    edge_type=EdgeType.DIRECTED,
)
BH_c = Edge(
    edge_id="BH",
    start_node=B,
    end_node=H,
    edge_type=EdgeType.DIRECTED,
)
BI_c = Edge(
    edge_id="BI",
    start_node=B,
    end_node=I,
    edge_type=EdgeType.DIRECTED,
)
HI_c = Edge(
    edge_id="HI",
    start_node=H,
    end_node=I,
    edge_type=EdgeType.UNDIRECTED,
)

# define factor functions


def phi_e(scope_product):
    "Visit to Asia factor p(a)"
    ss = set(scope_product)
    if ss == set([("E", True)]):
        return 0.01
    elif ss == set([("E", False)]):
        return 0.99
    else:
        raise ValueError("Unknown scope product")


def phi_fe(scope_product):
    "Tuberculosis | Visit to Asia factor p(t,a)"
    ss = set(scope_product)
    if ss == set([("F", True), ("E", True)]):
        return 0.05
    elif ss == set([("F", False), ("E", True)]):
        return 0.95
    elif ss == set([("F", True), ("E", False)]):
        return 0.01
    elif ss == set([("F", False), ("E", False)]):
        return 0.99
    else:
        raise ValueError("Unknown scope product")


def phi_dg(scope_product):
    "either tuberculosis or lung cancer | x ray p(e,x)"
    ss = set(scope_product)
    if ss == set([("D", True), ("G", True)]):
        return 0.98
    elif ss == set([("D", False), ("G", True)]):
        return 0.05
    elif ss == set([("D", True), ("G", False)]):
        return 0.02
    elif ss == set([("D", False), ("G", False)]):
        return 0.95
    else:
        raise ValueError("Unknown scope product")


def phi_a(scope_product):
    "smoke factor p(s)"
    ss = set(scope_product)
    if ss == set([("A", True)]):
        return 0.5
    elif ss == set([("A", False)]):
        return 0.5
    else:
        raise ValueError("Unknown scope product")


def phi_ab(scope_product):
    "smoke given bronchitis p(s,b)"
    ss = set(scope_product)
    if ss == set([("A", True), ("B", True)]):
        return 0.6
    elif ss == set([("A", False), ("B", True)]):
        return 0.3
    elif ss == set([("A", True), ("B", False)]):
        return 0.4
    elif ss == set([("A", False), ("B", False)]):
        return 0.7
    else:
        raise ValueError("Unknown scope product")


def phi_ac(scope_product):
    "lung cancer given smoke p(s,l)"
    ss = set(scope_product)
    if ss == set([("A", True), ("C", True)]):
        return 0.1
    elif ss == set([("A", False), ("C", True)]):
        return 0.01
    elif ss == set([("A", True), ("C", False)]):
        return 0.9
    elif ss == set([("A", False), ("C", False)]):
        return 0.99
    else:
        raise ValueError("Unknown scope product")


def phi_cdf(scope_product):
    "either tuberculosis or lung given lung cancer and tuberculosis p(e, l, t)"
    ss = set(scope_product)
    if ss == set([("C", True), ("D", True), ("F", True)]):
        return 1
    elif ss == set([("C", True), ("D", False), ("F", True)]):
        return 0
    elif ss == set([("C", False), ("D", True), ("F", True)]):
        return 1
    elif ss == set([("C", False), ("D", False), ("F", True)]):
        return 0
    elif ss == set([("C", True), ("D", True), ("F", False)]):
        return 1
    elif ss == set([("C", True), ("D", False), ("F", False)]):
        return 0
    elif ss == set([("C", False), ("D", True), ("F", False)]):
        return 0
    elif ss == set([("C", False), ("D", False), ("F", False)]):
        return 1
    else:
        raise ValueError("Unknown scope product")


def phi_ihb(scope_product):
    "cough, dyspnoea, bronchitis I, H, B p(c,d,b)"
    # In book this is phi(C,D,B) in table 6.4
    ss = set(scope_product)
    if ss == set([("H", True), ("I", True), ("B", True)]):
        return 16
    elif ss == set([("H", True), ("I", False), ("B", True)]):
        return 1
    elif ss == set([("H", False), ("I", True), ("B", True)]):
        return 4
    elif ss == set([("H", False), ("I", False), ("B", True)]):
        return 1
    elif ss == set([("H", True), ("I", True), ("B", False)]):
        return 2
    elif ss == set([("H", True), ("I", False), ("B", False)]):
        return 1
    elif ss == set([("H", False), ("I", True), ("B", False)]):
        return 1
    elif ss == set([("H", False), ("I", False), ("B", False)]):
        return 1
    else:
        raise ValueError("Unknown scope product")


def phi_hbd(scope_product):
    "cough, either tuberculosis or lung cancer, bronchitis D, H, B p(c,b,e)"
    # In book this is phi(C,B,E) in table 6.4
    ss = set(scope_product)
    if ss == set([("H", True), ("D", True), ("B", True)]):
        return 5
    elif ss == set([("H", True), ("D", False), ("B", True)]):
        return 2
    elif ss == set([("H", False), ("D", True), ("B", True)]):
        return 1
    elif ss == set([("H", False), ("D", False), ("B", True)]):
        return 1
    elif ss == set([("H", True), ("D", True), ("B", False)]):
        return 3
    elif ss == set([("H", True), ("D", False), ("B", False)]):
        return 1
    elif ss == set([("H", False), ("D", True), ("B", False)]):
        return 1
    elif ss == set([("H", False), ("D", False), ("B", False)]):
        return 1
    else:
        raise ValueError("Unknown scope product")


def phi_bd(scope_product):
    "bronchitis, either tuberculosis or lung cancer B, D p(b,e)"
    # In book this is phi(B,E) in table 6.4
    ss = set(scope_product)
    if ss == set([("B", True), ("D", True)]):
        return 1 / 90
    elif ss == set([("B", False), ("D", True)]):
        return 1 / 11
    elif ss == set([("B", True), ("D", False)]):
        return 1 / 39
    elif ss == set([("B", False), ("D", False)]):
        return 1 / 5
    else:
        raise ValueError("Unknown scope product")


# instantiate factors with factor functions and implied random
# variables in scope

E_cf = Factor(gid="E_cf", scope_vars=set([E]), factor_fn=phi_e)
EF_cf = Factor(gid="EF_cf", scope_vars=set([E, F]), factor_fn=phi_fe)
DG_cf = Factor(gid="DG_cf", scope_vars=set([D, G]), factor_fn=phi_dg)
A_cf = Factor(gid="A_cf", scope_vars=set([A]), factor_fn=phi_a)
AB_cf = Factor(gid="AB_cf", scope_vars=set([A, B]), factor_fn=phi_ab)
AC_cf = Factor(gid="AC_cf", scope_vars=set([A, C]), factor_fn=phi_ac)
CDF_cf = Factor(gid="CDF_cf", scope_vars=set([D, C, F]), factor_fn=phi_cdf)

IHB_cf = Factor(gid="IHB_cf", scope_vars=set([H, I, B]), factor_fn=phi_ihb)

HBD_cf = Factor(gid="HBD_cf", scope_vars=set([H, D, B]), factor_fn=phi_hbd)
BD_cf = Factor(gid="BD_cf", scope_vars=set([D, B]), factor_fn=phi_bd)


# instantiate lwf chain graph and make a query
cowell = LWFChainGraph(
    gid="cowell",
    nodes=set([A, B, C, D, E, F, G, H, I]),
    edges=set([AB_c, AC_c, CD_c, EF_c, FD_c, DG_c, DH_c, BH_c, BI_c, HI_c]),
    factors=set([E_cf, EF_cf, DG_cf, A_cf, AB_cf, AC_cf, CDF_cf, IHB_cf, HBD_cf, BD_cf]),
)
evidences = set([("E", True), ("A", True), ("G", False)])

final_factor, a = cowell.cond_prod_by_variable_elimination(set([B]), evidences)

val = round(final_factor.phi_normal(set([("B", True)])), 4)  # 0.60

pots = np.array([2.4455, 1.6303])  # from table 6.10
expected = pots[0] / np.sum(pots)

assert np.allclose(val, expected)
print(val, expected)

0.6 0.6000049070121203


In [6]:
print(a)
print(type(product_factor))

Factor: d69e2a81-63a9-4cd7-a573-c9eba4ef3b5f
Scope variables: {}Factor function: <function FactorOps.cls_sumout_var.<locals>.psi at 0x7fa168a00790>
<class 'pygmodels.pgmtype.factor.Factor'>


In [7]:
pots = np.array([2.4455, 1.6303])
print(pots[0] / np.sum(pots))

0.6000049070121203
